<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T165349Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da73e111bc4f4749f4a8520c56b9085abd9a3952517e4118ce55b2f987a2e24a89bd6198c4d649c840a75d21925bd792dc4c9b38b41dc02f784d27c16cf908b21d2229ff6001c413de4aafeb8491cd9542693ec62062654a9f53768ff1917a2832dd490cf4bc7525862a6c632495375f8a34724eea64eb5357d51e3aefe5776d08245c92669fc431e929f7f1a0f6bdf03e00d0181729d6427844703bec84eb56acd9f8ec282373e124de67bdf0c523dc52a8e384d5d896c4184aceed98fcb0afa368a4684e5e183a39fa95cab705732a5aa1c63f87a9e588741db8d51ecb432ad5ac730ea5d89444d717dc8d9fddc305a41b37a7ab0cc25c2535d4213d4eede62,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T165349Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D68363abefc107bf8c87775ca60ef7e940051b4a1cbe630394dfa1dc780990bcba3ec2db6a1bcbe19d2da6672b1a3491948aa5dfa98e9d2d9a7ca16b4a07167a6093b2f6c3f0e65b370cc457ed57ee2696291eda5adcb1f16ee953d9b8e6de0aca4c5b5505f894d3565893b7482a0a5cae4ef5bb9c4f632885836c86acf2c98b7252ee75f608593aac7dcd556ac37692b1780b787eb00c12b55d32f8b9892d5d47a39202094f686ea53d95191936656aabe2d67b0f21bb892f614ed49a2354ca78464dad33d376defb0ba10c050bdeb4bafe587e095ff007c02d9a98ede1e1eb16a63fea69a87d282ba59ea1d5a8c6909018a76bdad46d9e9c155acce769908a3,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T165350Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5a10d36cbe8b87108d11b740a973c803d5862035f5ea66ddd532caa70e2b6dc925be820f91d9169e9902b47124bc2ebf6414245501e27202b0d33dc5ed98c66925d57d54b63dab6c034fa76344f8d66957f06486dabcab0fa1ad8560c5fdbc163714385f2dddfbcd5f2637533ddab48b80cc83aebfeeaaf3f068c8ac75aff6187412faa2c267f5d472a50acc5bbb17de32b32934e085e4d8d0ec43e6003ba437ae75b8a57a7a6cd08bc5b6407e6ac66a7d0b1dbade9f2571e8d1135ff6e07c083e969d7250dd1f91691bf87bb6228a5e15f3766f73f2f253f06320034f628167a84558da0b86642ca1e68baee27f8f2bde8f906c263b3f85fc02c44ab37035ca'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powered by [Gemma 2B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the Python Enhancement Proposal ([PEPs](https://peps.python.org/)) documentation using a Retrival-Augmented Generation ([RAG](https://blogs.nvidia.com/blog/what-is-retrieval-augmented-generation/)) sponsored by a Chroma vectorial database using the open source embeddings [hkunlp/instructor-large](https://huggingface.co/hkunlp/instructor-large) of 768 entries, orchested by [langchaing](https://python.langchain.com/docs/use_cases/chatbots/) that is a framework that let you use differen models y les code changes

## Requirements

In [ ]:
!pip install langchain==0.0.209
!pip install chromadb==0.3.26
!pip install sentence-transformers==2.2.2
!pip install InstructorEmbedding==1.0.1

In [ ]:
#base
import os
import json
import re
import csv
import pandas as pd
from typing import *
# variables
from kaggle_secrets import UserSecretsClient
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup
# embeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
# vector database
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

# Classes

### Questions and answers data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        qa_data = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        qa_data = qa_data[qa_data['answer_score'] > 5]
        df_qa_data = qa_data[qa_data['Tag']=='python']
        return df_qa_data

    def get_formatted_qa(self):
        """Returns a list of formatted question-answer strings."""
        df_qa_data = self.qa_data()
        df_qa_data['Answer'] = df_qa_data['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        data = [
            f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data


### Python Enhancement Proposals data

In [ ]:
class Peps:
    """
    Scrapes Python Enhancement Proposals (PEPs) from https://peps.python.org/
    """

    def __init__(self):
        """Initiates the scraping process."""
        self.base_url = 'https://peps.python.org/'

    def scraper(self, url):
        """
        Fetches the HTML content of a given URL.

        Args:
            url (str): The URL to fetch.

        Returns:
            BeautifulSoup: A BeautifulSoup object representing the parsed HTML.
        """
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')

    def _fetch_pep_links(self):
        """Fetches links to individual PEP pages (private method).

        Returns:
            list: A list of PEP URLs.
        """
        soup = self.scraper(self.base_url)
        return list(
            set(
                [
                    self.base_url + ref['href']
                    for ref in soup.find_all('a', class_='pep reference internal')
                ]
            )
        )

    def _download_peps(self):
        """Downloads the content of individual PEPs (private method).

        Returns:
            list: A list of BeautifulSoup objects representing individual PEPs.
        """
        pep_links = self._fetch_pep_links()
        return [self.scraper(pep_link) for pep_link in pep_links]

    def scrape(self):
        """Extracts the relevant text content from each PEP.

        Returns:
            list: A list of text strings, each representing the content of a PEP.
        """
        pep_soups = self._download_peps()
        return [
            '\n'.join([word.text for word in soup.find_all('section')])
            for soup in pep_soups
        ]

    def jsonl_format(self):
        """Format the spcraped data to a jsonl format is it alist of dictionaries
        Returns:
            list: A list of dictionaries with the following:
                page_content: The content of each section scraped
                source: The linke where it was scraped
        """
        source_links = self._fetch_pep_links()
        pep_content = self.scrape()
        pep_data = dict(zip(source_links,pep_content))
        return [
            {
                'text': value,
                'source': key
            }
            for key, value in pep_data.items()
        ]

### Embeddings

In [ ]:
class Embeder:
    """
    Creates embeddings (numerical representations) of text documents for various tasks like semantic search and
    similarity comparison. Provides flexibility to choose between a large model for more comprehensive
    embeddings or a smaller, faster model.

    Attributes:

    large (bool): Flag indicating whether to use the large embedding model (default: True).
    model (str): Name of the embedding model to use.
    device (str): Device to use for computations (default: "cuda").
    """
    def __init__(
            self,
            large: bool = True
    ):
        """
        init(self, large: bool = True) -> None
            Initializes the class with the specified model size preference.
        """
        self.large = large
        if self.large:
            self.model = "hkunlp/instructor-large"
            self.device: str = "cuda"
        else:
            self.model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    def instructor(self):
        if self.large:
            embed = HuggingFaceInstructEmbeddings(
                model_name=self.model,
                model_kwargs={"device": self.device}
            )
        else:
            embed = SentenceTransformerEmbeddings(
                model_name=self.model
            )
        return embed

    def run(
            self,
            docs_list: list,
    ):
        """
        run(self, docs_list: list) -> List[List[float]]:
            Generates embeddings for the provided list of documents.
            Returns a list of lists, where each inner list represents the embedding vector for a document.
        """
        if docs_list is None:
            docs_list = ['']
        embed = self.instructor()

        return embed.embed_documents(docs_list)

In [ ]:
class Settings:
    user_secrets = UserSecretsClient()
    CHUNK_SIZE = user_secrets.get_secret("CHUNK_SIZE")
    CHUNK_OVERLAP = user_secrets.get_secret("CHUNK_OVERLAP")
    CHROMA_NAME_INDEX = user_secrets.get_secret("CHROMA_NAME_INDEX")
    K = user_secrets.get_secret("K")
    NN_THRESHOLD = user_secrets.get_secret("NN_THRESHOLD")

In [ ]:
embeder = Embeder()
settings = Settings()
class Retriever:
    """ retriever
    Retrieves the embeddings from a vectorial database
    """
    def __init__(self, data: list[dict]):
        """ initialize the retriever
        """
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(settings.CHUNK_SIZE),
            length_function=len,
            chunk_overlap=int(settings.CHUNK_OVERLAP)
        )
        self.index_name = settings.CHROMA_NAME_INDEX
        self.data = data

    def load(self):
        """
        Loads documents from the initialized data and returns them as a list of Document objects.
        """
        documents = []
        for obj in self.data:
            page_content = obj.get("text", "")
            metadata = {
                "source": obj.get("source", "local")
            }
            documents.append(Document(page_content=page_content, metadata=metadata))
        return documents

    def set(self):
        """
         update the vectorial database
        """
        data = self.load()
        splitter = self.text_splitter
        documents = splitter.split_documents(data)
        instructor = embeder.instructor()
        vector_db = Chroma.from_documents(
            documents=documents,
            embedding=instructor,
            persist_directory=self.index_name
        )
        vector_db.persist()

    def query(
            self,
            message: str,
            k: int = int(settings.K),
            threshold: float = float(settings.NN_THRESHOLD)
    ):
        """ retrieve the 2 more similar text chunks based on the message given
        """
        vectorial_db = Chroma(
            embedding_function=embeder.instructor(),
            persist_directory=self.index_name
        )
        res = vectorial_db.similarity_search_with_score(message, k=k)
        return [vector[0].page_content for vector in res if vector[1] < threshold]

## Data

### RAG Data

In [ ]:
%%time
peps = Peps()
peps_corpus = peps.scrape()
print(
    'corpus lenght:',
    len(peps_corpus),
    '\nsection <n> lenght:',
    len(peps_corpus[0]),
    '\nsample text:\n\n',
    peps_corpus[10][1306:2000]
)

In [ ]:
%%time
peps = Peps()
data = peps.jsonl_format()
data[-1]['text'][:2000]

### fine-tunning data

In [ ]:
%%time
PythonQAData = pythonQAData()
python_context = PythonQAData.get_formatted_qa()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1][1306:2000]
)

## RAG

In [ ]:
%%time
embeder = Embeder()
vec = embeder.run(["hello I'm goku"])[0]
print(len(vec))
vec[:10]

In [ ]:
%%time
retriever = Retriever(data)
retriever.set()

In [ ]:
retriever = Retriever(data)
question = "what is a List comprehensions and what is it's relation with map() and filter()?"
retriever.query(question)

## Chatbot

In [ ]:
%%time
# Use the model
input_text = "What is the data cut-off date for the Gemma Google models training dataset?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=200)
print(tokenizer.decode(outputs[0]))

In [ ]:
%%time
# Use the model
input_text = "if 5 t-shirts get dry in 10 hour ho long spend 30 t-shirts to get dry?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0